In [1]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import torch.nn.functional as F
import cv2
import numpy as np
import torch.utils.data as Data
from PIL import  Image,ImageDraw
import numpy as np
import random

Batch_Size = 50
num_epochs = 100
learning_rate = 1e-5
samples = 10000

data1 = []
data2= []


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% --  Generate dataset
im_drone = Image.open('drone4-1.png')
im_drone = im_drone.resize((35,35))

for i in range(samples):
    
    switch = random.randint(0,1)
    if switch==0:
        img = Image.new('RGB', (100, 100), color = 'white')
        #img = np.reshape(img,(28,28,1))
        #print(np.shape(img))
        row,col,ch= np.shape(img)
        mean = random.randint(1,6)
        var = random.random()
        sigma = var**0.5
        gauss_pos1 = np.random.normal(mean,sigma,(row,col,ch))
        noisy_pos1 = img + gauss_pos1
        im2_pos1 = Image.fromarray(noisy_pos1,'RGB')
        #draw = ImageDraw.Draw(im2_pos1)
        #draw.rectangle([20,20,40,40],fill='red',outline='red')
        #im2_pos1.show()
        #print(np.shape(np.transpose(np.asarray(im2_pos1),(2,0,1))))
        im2_pos1.paste(im_drone,(10,10),im_drone)
        im2_pos1 = np.reshape(im2_pos1,(100, 100,3))
        #print(np.shape(im2_pos1))
        data1.append(np.array(np.transpose(np.asarray(im2_pos1),(2,0,1)),dtype=np.float32))

        img_1 = Image.new('RGB', (100, 100), color = 'white')
        row_1,col_1,ch_1= np.shape(img_1)
        mean_1 = random.randint(1,6)
        var_1 = random.random()
        sigma_1 = var_1**0.5
        gauss_pos2 = np.random.normal(mean_1,sigma_1,(row_1,col_1,ch_1))
        noisy_pos2 = img_1 + gauss_pos2
        im2_pos2 = Image.fromarray(noisy_pos2,'RGB')
        #draw_1 = ImageDraw.Draw(im2_pos2)
        #draw_1.rectangle([20,20,40,40],fill='red',outline='red')
        im2_pos2.paste(im_drone,(10,10),im_drone)
        im2_pos2 = np.reshape(im2_pos2,(100, 100,3))
        data2.append(np.array(np.transpose(np.asarray(im2_pos2),(2,0,1)),dtype=np.float32))
    elif switch==1:
        img_blank = Image.new('RGB', (100, 100), color = 'white')
        row_blank,col_blank,ch_blank= np.shape(img_blank)
        mean_blank = random.randint(1,6)
        var_blank = random.random()
        sigma_blank = var_blank**0.5
        gauss_blank_pos1 = np.random.normal(mean_blank,sigma_blank,(row_blank,col_blank,ch_blank))
        noisy_blank_pos1 = img_blank + gauss_blank_pos1
        im2_blank_pos1 = Image.fromarray(noisy_blank_pos1,'RGB')
        im2_blank_pos1 = np.reshape(im2_blank_pos1,(100, 100,3))
        data1.append(np.array(np.transpose(np.asarray(im2_blank_pos1),(2,0,1)),dtype=np.float32))

        img_blank_1 = Image.new('RGB', (100, 100), color = 'white')
        row_blank_1,col_blank_1,ch_blank_1= np.shape(img_blank_1)
        mean_blank_1 = random.randint(1,6)
        var_blank_1 = random.random()
        sigma_blank_1 = var_blank_1**0.5
        gauss_blank_pos2 = np.random.normal(mean_blank_1,sigma_blank_1,(row_blank_1,col_blank_1,ch_blank_1))
        noisy_blank_pos2 = img_blank_1 + gauss_blank_pos2
        im2_blank_pos2 = Image.fromarray(noisy_blank_pos2,'RGB')
        im2_blank_pos2 = np.reshape(im2_blank_pos2,(100, 100,3))
        data2.append(np.array(np.transpose(np.asarray(im2_blank_pos2),(2,0,1)),dtype=np.float32))


#%%%%%%%%%%%%%%%%%%%%%%%%%%% Noisy data
img = Image.new('RGB', (100,100), color = 'white')
row,col,ch= np.shape(img)
mean = 0
var = 1
sigma = var**0.5
gauss = np.random.normal(mean,sigma,(row,col,ch))
noisy = img + gauss
im2 = Image.fromarray(noisy,'RGB')
im2 = np.reshape(im2,(100,100,3))
b = np.asarray(im2)
b = np.array(b, dtype=np.float32)
b = np.transpose(b,(2,0,1))
b = torch.from_numpy(b)

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%       

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Data Preparation

y0 = np.asarray(data1)
y1 = np.asarray(data2)



print(np.shape(y0))

x = torch.from_numpy(y0)
y = torch.from_numpy(y1)


torch_dataset = Data.TensorDataset(x,y)


loader = Data.DataLoader(

    dataset=torch_dataset,

    batch_size=Batch_Size,

    shuffle=True,

    num_workers=0,

)


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Defining NN

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()

        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()

        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(20000, 1)
        self.sigmoid1 = nn.Sigmoid()

    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)

        # Max pool 1
        out = self.maxpool1(out)

        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)

        # Max pool 2 
        out = self.maxpool2(out)

        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        out = self.sigmoid1(out)
        return out
    
model = CNNModel()


optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Loss Function

def h_score(fx, gy):
 
    fx = fx - fx.mean(0)

    gy = gy - gy.mean(0)

    Nsamples = fx.size(0)
    #print(fx, np.shape(fx))
    #print(gy, np.shape(gy))
    #print(np.shape((torch.transpose(fx,3,2))),np.shape(fx))
    covf = torch.matmul(fx.t(), fx) / Nsamples

    covg = torch.matmul(gy.t(), gy) / Nsamples

    h = -2 * torch.mean((fx * gy).sum(1)) + (covf * covg).sum()

    return h


optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-4)





for epoch in range(num_epochs):
    for x,y in loader:
        optimizer.zero_grad()
        # ===================forward=====================
        #loss = criterion(output1, img)
        loss = h_score(model(x),model(y))
        # ===================backward====================

        #optimizer_1.zero_grad()
        loss.backward()
        optimizer.step()
        #optimizer_1.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.item()))
    print("x: ", model(x[0].reshape(1,3,100, 100)))
    print("y:", model(y[0].reshape(1,3,100, 100)))
    print("b:", model(b.reshape(1,3,100, 100)))


(10000, 3, 100, 100)
epoch [1/100], loss:0.0000
x:  tensor([[6.3466e-15]], grad_fn=<SigmoidBackward>)
y: tensor([[4.6523e-13]], grad_fn=<SigmoidBackward>)
b: tensor([[1.0033e-16]], grad_fn=<SigmoidBackward>)
epoch [2/100], loss:0.0000
x:  tensor([[1.1600e-09]], grad_fn=<SigmoidBackward>)
y: tensor([[7.0742e-11]], grad_fn=<SigmoidBackward>)
b: tensor([[4.0244e-14]], grad_fn=<SigmoidBackward>)
epoch [3/100], loss:0.0000
x:  tensor([[2.8094e-08]], grad_fn=<SigmoidBackward>)
y: tensor([[3.9188e-09]], grad_fn=<SigmoidBackward>)
b: tensor([[7.1160e-12]], grad_fn=<SigmoidBackward>)
epoch [4/100], loss:0.0000
x:  tensor([[1.7895e-07]], grad_fn=<SigmoidBackward>)
y: tensor([[1.0632e-07]], grad_fn=<SigmoidBackward>)
b: tensor([[5.8685e-10]], grad_fn=<SigmoidBackward>)
epoch [5/100], loss:0.0000
x:  tensor([[1.5137e-06]], grad_fn=<SigmoidBackward>)
y: tensor([[1.1956e-05]], grad_fn=<SigmoidBackward>)
b: tensor([[2.5057e-08]], grad_fn=<SigmoidBackward>)
epoch [6/100], loss:0.0000
x:  tensor([[2.35

epoch [45/100], loss:-0.0000
x:  tensor([[1.6669e-07]], grad_fn=<SigmoidBackward>)
y: tensor([[2.0487e-07]], grad_fn=<SigmoidBackward>)
b: tensor([[7.7264e-09]], grad_fn=<SigmoidBackward>)
epoch [46/100], loss:-0.0000
x:  tensor([[2.7778e-07]], grad_fn=<SigmoidBackward>)
y: tensor([[8.2828e-08]], grad_fn=<SigmoidBackward>)
b: tensor([[1.2084e-08]], grad_fn=<SigmoidBackward>)
epoch [47/100], loss:-0.0000
x:  tensor([[4.9376e-07]], grad_fn=<SigmoidBackward>)
y: tensor([[3.4495e-07]], grad_fn=<SigmoidBackward>)
b: tensor([[1.9506e-08]], grad_fn=<SigmoidBackward>)
epoch [48/100], loss:0.0000
x:  tensor([[1.0607e-06]], grad_fn=<SigmoidBackward>)
y: tensor([[8.3302e-07]], grad_fn=<SigmoidBackward>)
b: tensor([[3.2446e-08]], grad_fn=<SigmoidBackward>)
epoch [49/100], loss:-0.0000
x:  tensor([[1.6901e-07]], grad_fn=<SigmoidBackward>)
y: tensor([[7.4383e-07]], grad_fn=<SigmoidBackward>)
b: tensor([[5.5564e-08]], grad_fn=<SigmoidBackward>)
epoch [50/100], loss:-0.0000
x:  tensor([[1.1554e-06]], 

epoch [90/100], loss:-0.4320
x:  tensor([[1.0000]], grad_fn=<SigmoidBackward>)
y: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
b: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
epoch [91/100], loss:-0.4351
x:  tensor([[1.0000]], grad_fn=<SigmoidBackward>)
y: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
b: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
epoch [92/100], loss:-0.4369
x:  tensor([[1.0000]], grad_fn=<SigmoidBackward>)
y: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
b: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
epoch [93/100], loss:-0.4369
x:  tensor([[1.]], grad_fn=<SigmoidBackward>)
y: tensor([[1.]], grad_fn=<SigmoidBackward>)
b: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
epoch [94/100], loss:-0.4375
x:  tensor([[1.0000]], grad_fn=<SigmoidBackward>)
y: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
b: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
epoch [95/100], loss:-0.4369
x:  tensor([[1.3474e-05]], grad_fn=<SigmoidBackward>)
y: tensor([[8.6029e-08]], grad_fn=<Sigmo

In [3]:
data3=[]

for i in range(1):
    img = Image.new('RGB', (100, 100), color = 'white')
    #img = np.reshape(img,(28,28,1))
    #print(np.shape(img))
    row,col,ch= np.shape(img)
    mean = random.randint(1,6)
    var = random.random()
    sigma = var**0.5
    gauss_pos1 = np.random.normal(mean,sigma,(row,col,ch))
    noisy_pos1 = img + gauss_pos1
    im2_pos1 = Image.fromarray(noisy_pos1,'RGB')
    #draw = ImageDraw.Draw(im2_pos1)
    #draw.rectangle([20,20,40,40],fill='red',outline='red')
    #im2_pos1.show()
    #print(np.shape(np.transpose(np.asarray(im2_pos1),(2,0,1))))
    
    im2_pos1.paste(im_drone,(10,10),im_drone)
    im2_pos1.show()
    im2_pos1 = np.reshape(im2_pos1,(100,100,3))

    data3.append(np.array(np.transpose(np.asarray(im2_pos1),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    print("x: ", model(x_1[i].reshape(1,3,100,100)))

x:  tensor([[2.1673e-08]], grad_fn=<SigmoidBackward>)
